# Proteus

## [ProteusV0.3](https://huggingface.co/dataautogpt3/ProteusV0.3)

## Check for CUDA/MPS Device

In [1]:
import torch
from diffusers import (
    StableDiffusionXLPipeline, 
    KDPM2AncestralDiscreteScheduler,
    AutoencoderKL
)

if torch.cuda.is_available():
    device = torch.device("cuda")
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    print ("CUDA & MPS devices not found.")

print("Testing torch device")
torch.ones(2, device=device) 

/Users/captain2mac/Documents/Code/HuggingFace/ml_env/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/captain2mac/Documents/Code/HuggingFace/ml_env/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/captain2mac/Documents/Code/HuggingFace/ml_env/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/captain2mac/Documents/Code/HuggingFace/ml_env/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytr

Testing torch device


tensor([1., 1.], device='mps:0')

## Loading the VAE autoencoder

In [2]:
# Load VAE component
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", 
    torch_dtype=torch.float16
)

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

## Configuring the pipeline to be used
### Configuring the `KDPM2AncestralDiscreteScheduler` schedular for pipeline the piping to `device`

In [7]:
# Configure the pipeline
model_v2 = "dataautogpt3/ProteusV0.2"
model_v3 = "dataautogpt3/ProteusV0.3"
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_v3, 
    vae=vae,
    torch_dtype=torch.float16
)
pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)
pipe

text_encoder/model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.24.0",
  "_name_or_path": "dataautogpt3/ProteusV0.2",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "KDPM2AncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

## Actual prompting

### Inputs:
- `prompt`: The prompt to the pipeline
- `negative_prompt `: What you don't wanna see
- `cfg_scale`: 8 or 7
- `guidance_scale`: Higher will be closer to prompt but quality may be affected
  - typically around 7.5 to 10
- `num_inference_steps`: 20 to 60, higher is slower but greater quality.
- `images_per_prompt`: Number of output images
- `height`: height of output image (should be divisible by 8)
- `width`: width of output image (should be divisible by 8)

### Ouput:
- array of pillow images

In [33]:
# Define prompts and generate image
prompt = "Generate a visually striking Hindu OM symbol tailored for a \"Apple Watch\" wallpaper, featuring vibrant colors inspired by mountains and \"Lord Shiva\" set against a cinematic background"
negative_prompt = "nsfw, bad quality, bad anatomy, worst quality, low quality, low resolutions, extra fingers, blur, blurry, ugly, wrongs proportions, watermark, image artifacts, lowres, ugly, jpeg artifacts, deformed, noisy image"
cfg_scale = 8
guidance_scale = 10
num_inference_steps = 42
images_per_prompt = 4
(width, height) = (512, 608)

images = pipe(
    prompt, 
    negative_prompt=negative_prompt, 
    width=width,
    height=height,
    num_images_per_prompt=images_per_prompt,
    guidance_scale=guidance_scale,
    cfg_scale=cfg_scale,
    num_inference_steps=num_inference_steps
).images

images

  0%|          | 0/42 [00:00<?, ?it/s]

[<PIL.Image.Image image mode=RGB size=512x608>,
 <PIL.Image.Image image mode=RGB size=512x608>,
 <PIL.Image.Image image mode=RGB size=512x608>,
 <PIL.Image.Image image mode=RGB size=512x608>]

### Saving Images

In [34]:
from time import sleep
from datetime import datetime

for image in images:
    now = datetime.now()
    current = now.strftime("%H-%M-%S")
    print(current)
    image.save(f"../outputs/om{current}.png")
    sleep(1)

17-31-39
17-31-41
17-31-42
17-31-43
